# <center>
<div style="
    background: #f0f333ff;
    border-left: 5px solid #ecd242ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h1 style="
        text-align: center;
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 600;
    ">
    Test
    </h1>
</div>

##
<div style="
    background: #40f0aa;
    border-left: 5px solid #0c7230ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h2 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Constraints
    </h2>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    add_bool_and
    </h3>
</div>

In [ ]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

# Variables booléennes
x = model.NewBoolVar('x')
y = model.NewBoolVar('y')

# z = (x ET y)
model.AddBoolAnd([x, y])

# Solveur
solver = cp_model.CpSolver()
solver.Solve(model)

for xv in [0, 1]:
    for yv in [0, 1]:
        model.Add(x == xv)
        model.Add(y == yv)
        solver.Solve(model)
        print(f"x={xv}, y={yv} → z={solver.Value(z)}")

In [28]:
from ortools.sat.python import cp_model

model = cp_model.CpModel()

# Variables booléennes
x = model.NewBoolVar('x')
y = model.NewBoolVar('y')
z = model.NewBoolVar('z')

# z = (x ET y)
model.AddBoolAnd([x, y]).OnlyEnforceIf(z)
model.AddBoolOr([x.Not(), y.Not()]).OnlyEnforceIf(z.Not())

# Solveur
solver = cp_model.CpSolver()
solver.Solve(model)

# Affichage des 8 combinaisons possibles
for xv in [0, 1]:
    for yv in [0, 1]:
        model.Add(x == xv)
        model.Add(y == yv)
        solver.Solve(model)
        print(f"x={xv}, y={yv} → z={solver.Value(z)}")


x=0, y=0 → z=0
x=0, y=1 → z=0
x=1, y=0 → z=0
x=1, y=1 → z=0


###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Add constraints
    </h3>
</div>

###
<div style="
    background: #9feaf2ff;
    border-left: 5px solid #1d28c1ff;
    padding: 15px 25px;
    margin: 20px 0;
    box-shadow: 0 2px 8px rgba(0,0,0,0.08);
">
    <h3 style="
        color: #2e3a59;
        font-family: 'Segoe UI', sans-serif;
        margin: 0;
        font-weight: 500;
    ">
    Second Test : Real situation
    </h3>
</div>

In [ ]:
from ortools.sat.python import cp_model

# -------------------------
# Paramètres du problème
# -------------------------
employees = ["Alice", "Bob", "Cory", "Dan"]
days = ["Lundi", "Mardi", "Mercredi", "Jeudi", "Vendredi"]
activities = ["Tél", "Dérog", "Récla"]

n_emp = len(employees)
n_days = len(days)

# -------------------------
# Modèle
# -------------------------
model = cp_model.CpModel()

# x[e, d, s, a] = 1 si l'employé e fait l'activité a le jour d et shift s
x = {}
for e in employees:
    for d in days:
        for a in activities:
            x[e, d, a] = model.NewBoolVar(f"x_{e}_{d}_{a}")

# -------------------------
# Contraintes
# -------------------------

for a in activities:
    for d in days:
        model.add_exactly_one(x[(e, d, a)] for e in employees)

for d in days:
    model.add_bool_and([x[(e, d, activities[0])],
                        x[(e, d, activities[1])],
                        x[(e, d, activities[2])]] for e in employees)

# -------------------------
# Solveur
# -------------------------
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 10
solver.parameters.num_search_workers = 8

result = solver.Solve(model)

# -------------------------
# Affichage du planning
# -------------------------
if result == cp_model.OPTIMAL or result == cp_model.FEASIBLE:
    print("\t        " + " ".join([f"{a:4}" for a in activities]))
    for d in days:
        st = f"{d:10} →   "
        for e in employees:
            for a in activities:
                if solver.Value(x[e, d, a]) == 1:
                    st += (f"{e} ")
        print(st)
else:
    print("❌ Aucune solution trouvée.")